# Objects

- Classes are used to construct objects and support inheritance
- Classes are not types
- Objects have _object types_ enclosed in angle brackets < ... >, containing just types of the methods. Fields are not part of the public interface of an object.
- All the interactions with an object is through its methods
- Unlike functions, methods can have zero paramaters and are invoked with the use of # character e.g. ```counter#incr```

In [11]:
let counter = object
    val mutable count = 0
    method count = count 
    method incr = count <- count + 1
    method decr = count <- count - 1
end ;;

counter#count ;;
counter#incr ;;
counter#count ;;
counter#decr ;;
counter#count ;;

val counter : < count : int; decr : unit; incr : unit > = <obj>


- : int = 0


- : unit = ()


- : int = 1


- : unit = ()


- : int = 0


- Methods can be used without an explicit type declaration
- Object types are inferred automatically
- The .. in object types means the types are open. 
  ```< to_string : 'a; .. >``` specifies an object that must have at least ```to_string``` method, and possibly some others as well
- The .. is a special kind of type variable called a row variable and typing scheme using row variables is called _row polymorphism_

In [19]:
let show e = e#to_string ;;
show (object method to_string = "hello" end) ;;
let student = object
    method name = "foo"
    method age = 12
    method to_string = "I am foo."
end ;;
show foo ;;

val show : < to_string : 'a; .. > -> 'a = <fun>


- : string = "hello"


val student : < age : int; name : string; to_string : string > = <obj>


- : string = "I am foo."


- Closing an object type

In [17]:
let bark (dog: <bark : string>) = dog#bark ;;
bark (object method bark = "wof wof" end) ;;

val bark : < bark : string > -> string = <fun>


- : string = "wof wof"


<br/>

##### 🟢 &nbsp; &nbsp; Immutable Objects
The expression {< ... >} produces a copy of the current object, with the same type, and the specified fields updated.

In [35]:
let point2d = object
    val _x = 0.0
    val _y = 0.0
    method x = _x
    method y = _y
    method add x y = {< _x = x; _y = y >}
    method to_string = Printf.sprintf "x = %F, y = %F" _x _y
end ;;

let p = point2d#add 1.1 2.2 ;;
p#x ;;
p#y ;;
p#to_string ;;
point2d#x ;;
point2d#y ;;

val point2d :
  < add : float -> float -> 'a; to_string : string; x : float; y : float >
  as 'a = <obj>


val p :
  < add : float -> float -> 'a; to_string : string; x : float; y : float >
  as 'a = <obj>


- : float = 1.1


- : float = 2.2


- : string = "x = 1.1, y = 2.2"


- : float = 0.


- : float = 0.


<br/>

##### 🟢 &nbsp; &nbsp; Width Subtyping
An object type D is a subtype of P if D has all the methods of P. ```D :> P```

In [42]:
type control = < name : string; location : float * float > ;;
let create_textbox ~(name: string) ~(location: float * float) = object
    val mutable _name = name
    val mutable _location = location
    val mutable _text = name 
    method name = _name
    method set_name name = _name <- name 
    method location = _location
    method set_location p = _location <- p
    method text = _text
    method set_text text = _text <- text 
end ;;

let show (c:control) = Printf.sprintf "name = %s, location = (%F, %F)" c#name (fst c#location) (snd c#location) ;;
let txtbox = create_textbox ~name:"MyTextBox" ~location:(0., 0.) ;;
txtbox#set_location (11., 11.) ;;
txtbox#set_text "Text Box" ;;
show (txtbox :> control) ;;

type control = < location : float * float; name : string >


val create_textbox :
  name:string ->
  location:float * float ->
  < location : float * float; name : string;
    set_location : float * float -> unit; set_name : string -> unit;
    set_text : string -> unit; text : string > =
  <fun>


val show : control -> string = <fun>


val txtbox :
  < location : float * float; name : string;
    set_location : float * float -> unit; set_name : string -> unit;
    set_text : string -> unit; text : string > =
  <obj>


- : unit = ()


- : unit = ()


- : string = "name = MyTextBox, location = (11., 11.)"


<br/>

##### 🟢 &nbsp; &nbsp; Depth Subtyping
An object type ```< m : d >``` is a subtype of ```< m : p >``` if `d` is a subtype of `p`.

In [61]:
type control = < name : string; location : float * float > ;;
let create_textbox ~(name: string) ~(location: float * float) = object
    val mutable _name = name
    val mutable _location = location
    val mutable _text = name 
    method name = _name
    method set_name name = _name <- name 
    method location = _location
    method set_location p = _location <- p
    method text = _text
    method set_text text = _text <- text 
end ;;

let create_button ~(name: string) ~(location: float * float) = object
    val mutable _name = name
    val mutable _location = location
    val mutable _content = None 
    method name = _name
    method set_name name = _name <- name 
    method location = _location
    method set_location p = _location <- p
    method content = _content
    method set_content (c:control) = _content <- Some c  
end ;;

let text_component = object
    method render = create_textbox ~name:"user name" ~location:(1., 1.)     
end ;;

let submit_component = object
    method render = create_button ~name:"submit" ~location:(10., 10.)   
end ;;

type component = < render : control > ;;
let components = [| (text_component :> component); (submit_component :> component) |] ;;

(Array.get components 0)#render#name ;;
(Array.get components 1)#render#name ;;

type control = < location : float * float; name : string >


val create_textbox :
  name:string ->
  location:float * float ->
  < location : float * float; name : string;
    set_location : float * float -> unit; set_name : string -> unit;
    set_text : string -> unit; text : string > =
  <fun>


val create_button :
  name:string ->
  location:float * float ->
  < content : control option; location : float * float; name : string;
    set_content : control -> unit; set_location : float * float -> unit;
    set_name : string -> unit > =
  <fun>


val text_component :
  < render : < location : float * float; name : string;
               set_location : float * float -> unit;
               set_name : string -> unit; set_text : string -> unit;
               text : string > > =
  <obj>


val submit_component :
  < render : < content : control option; location : float * float;
               name : string; set_content : control -> unit;
               set_location : float * float -> unit;
               set_name : string -> unit > > =
  <obj>


type component = < render : control >


val components : component array = [|<obj>; <obj>|]


- : string = "user name"


- : string = "submit"


<br/>

##### 🟢 &nbsp; &nbsp; Row Polymorphism vs Subtyping

##### _Row Polymorphism_
- Row polymorphism is preferred with functions that can be applied to objects of different types. The return type of such functions can be built from the open object type of its argument, preserving additional methods that it may have.

In [74]:
let motorbike ~model ~color ~weight = object 
    val _model = model
    val _color = color
    val _weight = weight
    method model = _model
    method color = _color
    method weight = _weight
    method drive = Printf.sprintf "Driving a  Motorbike"
end ;;

let aeroplane ~model ~color ~weight ~engines = object
    val _model = model
    val _color = color
    val _weight = weight
    val _engines : float = engines
    method model = _model
    method color = _color
    method weight = _weight
    method engines = _engines
    method drive = Printf.sprintf "Driving an Aeroplane"
end ;;

let drive v = v#drive ;;
drive (motorbike ~model:"123" ~color:"blue" ~weight:20.) ;;
drive (aeroplane ~model:"ABC1" ~color:"silver" ~weight:2000. ~engines:4.) ;;

(* preserves more type information *)
let filter l =
    l |> List.filter (fun x -> x#color = "blue") ;;

filter [
    motorbike ~model:"123" ~color:"blue" ~weight:20. ;
    motorbike ~model:"221" ~color:"silver" ~weight:50. ;   
] ;;

val motorbike :
  model:'a ->
  color:'b ->
  weight:'c -> < color : 'b; drive : string; model : 'a; weight : 'c > =
  <fun>


val aeroplane :
  model:'a ->
  color:'b ->
  weight:'c ->
  engines:float ->
  < color : 'b; drive : string; engines : float; model : 'a; weight : 'c > =
  <fun>


val drive : < drive : 'a; .. > -> 'a = <fun>


- : string = "Driving a  Motorbike"


- : string = "Driving an Aeroplane"


val filter : (< color : string; .. > as 'a) list -> 'a list = <fun>


- : < color : string; drive : string; model : string; weight : float > list =
[<obj>]


val filter_sub : < color : string > list -> < color : string > list = <fun>


error: compile_error

<br/>

##### _Subtyping_
- To place different types of objects in the same container - heterogenous container
- To store different types of object in the same ref cell.

In [83]:
type vehicle = < model : string; color : string; weight : float; drive : string > ;;
let motorbike ~model ~color ~weight = object 
    val _model = model
    val _color = color
    val _weight = weight
    method model = _model
    method color = _color
    method weight = _weight
    method drive = Printf.sprintf "Driving a  Motorbike"
end ;;

let aeroplane ~model ~color ~weight ~engines = object
    val _model = model
    val _color = color
    val _weight = weight
    val _engines : float = engines
    method model = _model
    method color = _color
    method weight = _weight
    method engines = _engines
    method drive = Printf.sprintf "Driving an Aeroplane"
end ;;

(* don't preserve type information *)
let filter (l: <color : string> list) =
    l |> List.filter (fun x -> x#color = "blue") ;;

filter_sub [
    ((motorbike ~model:"123" ~color:"blue" ~weight:20.) :> <color : string>) ;
    ((motorbike ~model:"221" ~color:"silver" ~weight:50.) :> <color : string>) ;
    ((aeroplane ~model:"ABC1" ~color:"blue" ~weight:2000. ~engines:4.) :> <color : string>)
] ;;

(* ref cells *)
let r : vehicle ref = ref ((motorbike ~model:"123" ~color:"blue" ~weight:20.) :> vehicle) ;;
!r#drive ;;
r := ((aeroplane ~model:"ABC1" ~color:"blue" ~weight:2000. ~engines:4.) :> vehicle) ;;
!r#drive ;;

type vehicle =
    < color : string; drive : string; model : string; weight : float >


val motorbike :
  model:'a ->
  color:'b ->
  weight:'c -> < color : 'b; drive : string; model : 'a; weight : 'c > =
  <fun>


val aeroplane :
  model:'a ->
  color:'b ->
  weight:'c ->
  engines:float ->
  < color : 'b; drive : string; engines : float; model : 'a; weight : 'c > =
  <fun>


val filter : < color : string > list -> < color : string > list = <fun>


- : < color : string > list = [<obj>; <obj>]


val r : vehicle ref = {contents = <obj>}


- : string = "Driving a  Motorbike"


- : unit = ()


- : string = "Driving an Aeroplane"
